In [ ]:
import sys
sys.path.append("../libs/")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from utils import shift_data, print_hp,print_line, graficarTodo, split_df, graficarClases, plot_metrics, plot_cm, plot_roc, plot_prc, plot_probs
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
import datetime
import os
import keras_tuner as kt
from keras_tuner import HyperModel
import keras.metrics as metrics
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.losses import BinaryCrossentropy

# Carga de datos

In [ ]:
df = pd.read_csv('../data/PERU_DATA_DIFF.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.set_index('DATE')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
graficarTodo(df,'Features')

# Mostrar Imbalance de clases

In [ ]:
neg, pos = np.bincount(df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

In [ ]:
graficarClases(df['Class'])

In [ ]:
initial_bias = np.log([pos/neg])

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

# Dividir Datos

In [ ]:
target_col = ['Class']
features = df.columns[(df.columns!=target_col[0])]
df_train, df_test = split_df(df,0.3)

# Normalizar

In [ ]:
df_x_train = df_train[features].copy()
df_y_train = df_train[target_col].copy()
df_x_test = df_test[features].copy()
df_y_test = df_test[target_col].copy()

scaler = StandardScaler()

df_x_train.iloc[:,:] = scaler.fit_transform(df_x_train)
df_x_test.iloc[:,:] = scaler.transform(df_x_test)

df_x_train.iloc[:,:] = np.clip(df_x_train,-5,5)
df_x_test.iloc[:,:] = np.clip(df_x_test,-5,5)

# Construccion

## Modelo

In [ ]:
METRICS = [
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'), 
      metrics.BinaryAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
      metrics.AUC(name='auc'),
      metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def build_lstm(n_steps_in: int, n_features: int, lstm_cells: int,
                lstm_dropout : float, dense_layers : int, dense_nodes : int, 
                dense_dropout : float, learning_rate: float, dense_activation = None,
                metrics = METRICS, output_bias = None):
    if output_bias is not None:
        output_bias = Constant(output_bias)
    model = Sequential()
    model.add(LSTM(lstm_cells, input_shape=(n_steps_in, n_features)))
    model.add(Dropout(lstm_dropout))
    for i in range(dense_layers):
        model.add(Dense(dense_nodes, activation=dense_activation))
    model.add(Dropout(dense_dropout))
    model.add(Dense(1, activation='sigmoid', bias_initializer=output_bias))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss=BinaryCrossentropy(), metrics=metrics)
    return model

## Optimizador

In [ ]:
class LSTMHyperModel(HyperModel):

    def __init__(self, n_features_in,n_steps_out, output_bias = None, class_weight= None, name = None, tunable = True):
        super().__init__(name=name, tunable=tunable)
        self.n_features = n_features_in
        self.n_steps_out = n_steps_out
        self.output_bias = output_bias
        self.class_weight = class_weight

    def build(self, hp):
        # Parametrizamos nro de capas, nro de nodos y ratio de aprendizaje
        hp_time_steps = hp.Int('steps_in',4,10,step=1)
        hp_lstm_cells = hp.Int('lstm_cells',32,356,step=16)
        hp_lstm_dropout = hp.Float('lstm_dropout',0.1,0.5,step=0.1)
        hp_dense_layers = hp.Int('dense_layers',1,5,step=1)
        hp_dense_nodes = hp.Int('dense_nodes',32,356,step=16)
        hp_dense_dropout = hp.Float('dense_dropout',0.1,0.5,step=0.1)
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
        hp_dense_activation = hp.Choice('dense_activation', values=['relu','tanh'])

        return build_lstm(
            n_steps_in = hp_time_steps,
            n_features = self.n_features,
            lstm_cells = hp_lstm_cells,
            lstm_dropout = hp_lstm_dropout,
            dense_layers = hp_dense_layers,
            dense_nodes = hp_dense_nodes,
            dense_dropout = hp_dense_dropout,
            learning_rate = hp_learning_rate,
            dense_activation = hp_dense_activation
        )
        
    def fit(self, hp, model,x,y,**kwargs):
        x_s,y_s = shift_data(x, y, hp.get('steps_in'), self.n_steps_out)
        #mini_batch = 32
        batch_learning = len(x_s)
        return model.fit(x = x_s, y = y_s, batch_size = batch_learning, class_weight=self.class_weight, **kwargs)

In [ ]:
n_steps_out = [1,6,12,24]
n_features = len(features)
ajuste_path = os.path.normpath('G:/')
fecha_hora = datetime.datetime.now().strftime('%Y%m%d_%H%M')
objective = kt.Objective('val_prc',direction='max')
max_epochs = 30
n_epochs = 50

In [ ]:
output_path = 'ajustes/'
output_path_model = 'modelos/'
name_prj = 'LSTM_USA_'
N = 20


In [ ]:
def optimizar_y_entrenar(name_prj, n_steps_out):
    name_prj = name_prj + str(n_steps_out) +'_'+fecha_hora
    name_model_tun = output_path+name_prj+'.h5'
    # Condicion de parada: 10 epocas despues del menor val_loss
    es = EarlyStopping(
        monitor='val_prc', 
        verbose=0,
        patience=10,
        mode='max',
        restore_best_weights=True)
    
    # --------------------- Optimizacion --------------------------------------
    lstm_hypermodel = LSTMHyperModel(n_features, n_steps_out, class_weight = class_weight) # output_bias=initial_bias

    lstm_tuner = kt.Hyperband(
        lstm_hypermodel,
        objective = objective,
        max_epochs = max_epochs,
        #directory = ajuste_path,
        project_name = name_prj,
        overwrite=True)

    print("Optimizando...")
    lstm_tuner.search(x = df_x_train, y = df_y_train, validation_split = 0.5, epochs = n_epochs
        , verbose = 2, shuffle = False, callbacks = [es])

    # guardar parametros de mejor modelo
    best_lstm_hps = lstm_tuner.get_best_hyperparameters(num_trials = 1)[0]
    print_hp(output_path+name_prj+'.txt',lstm_tuner)

    lstm_model = lstm_tuner.hypermodel.build(best_lstm_hps)
    lstm_model.save(name_model_tun) # modelo sin entrenar
    print(best_lstm_hps.values)

    # --------------------- Entrenamiento --------------------------------------
    n_steps_in = best_lstm_hps.get('steps_in')
    x_train, y_train = shift_data(df_x_train, df_y_train, n_steps_in, n_steps_out)
    x_test, y_test = shift_data(df_x_test, df_y_test, n_steps_in, n_steps_out)
    batch_learning = len(x_train)

    prcs = []
    models = []
    res_path = output_path_model+name_prj+".csv"
    print_line("loss,tp,fp,tn,fn,accuracy,precision,recall,auc,prc\n",res_path)

    print("Entrenando...")
    for i in range(N):
        name_model = output_path_model+name_prj+'_'+str(i)+'.h5'
        # Condicion de parada: 10 epocas despues del menor val_loss
        es_t = EarlyStopping(
            monitor='val_prc', 
            verbose=0,
            patience=10,
            mode='max',
            restore_best_weights=True)
        models.append(name_model)

        history = lstm_model.fit(x=x_train, y=y_train, validation_split = 0.5, epochs = n_epochs
            , verbose = 0, shuffle = False, callbacks = [es_t], batch_size = batch_learning, class_weight=class_weight)

        lstm_model.save(name_model) 
        # --------------------- Evaluacion --------------------------------------
        res = lstm_model.evaluate(x_test, y_test,
                                        batch_size=batch_learning, verbose=0)
        print_line(f'{res[0]},{res[1]},{res[2]},{res[3]},{res[4]},{res[5]},{res[6]},{res[7]},{res[8]},{res[9]}\n', res_path)
        prcs.append(res[9]) #guarda metrica a comparar

    #--------- Evaluando el mejor ------------------------
    # print("Evaluando...")
    # best_model = models[np.argmax(prcs)]
    # print(f'\nMejor modelo: {best_model} con prc: {np.max(prcs)}\n')

    # lstm_model = load_model(best_model)
    # train_predictions = lstm_model(x_train) #, batch_size=batch_learning
    # test_predictions = lstm_model(x_test)

    # plot_cm(y_test, test_predictions)
    # plot_roc(y_train, train_predictions, y_test, test_predictions)
    # plot_prc(y_train, train_predictions, y_test, test_predictions)
    # plot_probs(y_train, train_predictions, 'Train')
    # plot_probs(y_test, test_predictions, 'Test')

# Modelo para 1 mes a futuro

In [ ]:
optimizar_y_entrenar(name_prj, n_steps_out[0])

# Modelo para 6 meses a futuro

In [ ]:
optimizar_y_entrenar(name_prj, n_steps_out[1])

# Modelo para 12 meses a futuro

In [ ]:
optimizar_y_entrenar(name_prj, n_steps_out[2])

# Modelo para 24 meses a futuro

In [ ]:
optimizar_y_entrenar(name_prj, n_steps_out[3])

In [ ]:
# #sys.path.append("../libs/")
# from notifications import enviar_correo
# enviar_correo("Ajuste de Parametros Finalizado!","Se ha completado: {}".format(name_prj))